# Interactive Visualisation Dashboard

In [ ]:
import pandas as pd
import numpy as np
import panel as pn
pn.extension("tabulator")
import matplotlib.pyplot as plt
import hvplot.pandas
import os


In [ ]:
# Get the script's directory
script_dir = os.path.dirname(os.path.abspath(__file__))
file_path = os.path.join(script_dir, "synthetic_data.csv")

df = pd.read_csv(file_path)
df

,Transportation partner,Date,Tons carried,Amount paid
0,Driver_A,2018-01-01,5621.780832,71505.42
1,Driver_A,2018-02-01,8123.957593,86057.36
2,Driver_A,2018-03-01,4092.130483,37542.86
3,Driver_A,2018-04-01,3406.585285,41673.67
4,Driver_A,2018-05-01,7207.805082,81806.05
...,...,...,...,...
395,Driver_E,2024-04-01,4073.459822,47839.27
396,Driver_E,2024-05-01,6541.220752,78332.77
397,Driver_E,2024-06-01,3126.771287,41163.31
398,Driver_E,2024-07-01,9524.827977,109644.55


*****Preprocessing*****

In [3]:
df.columns = ["Transportation partner", "Date", "Tons carried", "Amount paid"]
df['Date'] = pd.to_datetime(df['Date']) 
df.dtypes

Transportation partner            object
Date                      datetime64[ns]
Tons carried                     float64
Amount paid                      float64
dtype: object

In [4]:
df = df.fillna(0)

In [5]:
idf = df.interactive()

***1. Tons carried and amount paid by partner over time (line graph)***

In [6]:
dates = pd.date_range(start="2018-01-01", end="2024-08-01", freq='MS')

months = [date.strftime('%Y-%m') for date in dates]

month_slider = pn.widgets.DiscreteSlider(name="Month Slider", options=months, value=months[0])
month_slider

BokehModel(combine_events=True, render_bundle={'docs_json': {'1e63ab05-4293-41c4-8f0c-dd5337b702a5': {'version…

In [7]:
# radio buttons
yaxis = pn.widgets.RadioButtonGroup(
    name = "Y axis",
    options = ["Tons carried", "Amount paid"],
    button_type = "success"
)

In [8]:
pipeline1 = (
    idf[
        (idf.Date <= month_slider)
    ]
    .groupby(["Date", "Transportation partner"])[yaxis].mean()
    .to_frame()
    .reset_index()
    .sort_values(by="Date")
    .reset_index(drop=True)    
)

In [9]:
pipeline1

BokehModel(combine_events=True, render_bundle={'docs_json': {'bde2722c-28cb-452c-9370-1a07da5c3100': {'version…

In [10]:
plot1 = pipeline1.hvplot(x = "Date", by = "Transportation partner", y = yaxis, line_width = 3, title = "Tons carried/ Amount paid by partner")
plot1

BokehModel(combine_events=True, render_bundle={'docs_json': {'21a51d2e-648f-455c-8269-2b7506efd1c2': {'version…

***2. Tons carried/ Amount paid over time by partner (Table chart)***

In [11]:
table = pipeline1.pipe(pn.widgets.Tabulator, pagination = "remote", page_size = 10, sizing_mode = "stretch_width")
table

BokehModel(combine_events=True, render_bundle={'docs_json': {'d0640d6f-47da-42ea-9058-6c176658c916': {'version…

## ***3. Tons carried/ Amount paid (scatterplot)***

In [12]:
pipeline2 = (
    idf[
        (idf.Date == month_slider)
    ]
    .groupby(["Date", "Transportation partner", "Tons carried"])["Amount paid"].mean()
    .to_frame()
    .reset_index()
    .sort_values(by="Date")
    .reset_index(drop=True)    
)

In [13]:
pipeline2

BokehModel(combine_events=True, render_bundle={'docs_json': {'619a4ba7-f6af-4d0f-81f2-4f9422930fd2': {'version…

In [14]:
plot2 = pipeline2.hvplot(x = "Tons carried", 
                                                                y = "Amount paid", 
                                                                by = "Transportation partner", 
                                                                size = 80,
                                                                kind = "scatter",
                                                                alpha = 0.7,
                                                                legend = False,
                                                                height = 500,
                                                                width = 500)
plot2

BokehModel(combine_events=True, render_bundle={'docs_json': {'227a01e8-c411-4cd5-a3a0-40846d0b5b9e': {'version…

***4. Tons carried/ Amount paid by partner (bar chart)***

In [15]:
# radio buttons 
yaxis2 = pn.widgets.RadioButtonGroup(
    name = "Y axis",
    options = ["Tons carried", "Amount paid"],
    button_type = "success"
)

pipeline3 = (
    idf[
        (idf.Date == month_slider)
    ]
    .groupby(["Date", "Transportation partner"])[yaxis2].sum()
    .to_frame()
    .reset_index()
    .sort_values(by="Date")
    .reset_index(drop=True)    
)

In [ ]:
plot3 = pipeline3.hvplot(kind = "bar",
                            x = "Transportation partner",
                            y = yaxis2,
                            title = "Tons carried/ Amount paid by partner")
plot3

BokehModel(combine_events=True, render_bundle={'docs_json': {'0eff7f0e-cc19-4910-be51-3f4087bddb15': {'version…

***Creating dashboard***

In [ ]:
# Layout using template
template = pn.template.FastListTemplate(
    title = "Transportation partner dashboard",
    sidebar = [pn.pane.Markdown("# Compare different Transportation Partners"),
        #       pn.pane.JPG("LKWWALTER.jpg", sizing_mode = "scale_both"),
               #pn.pane.Markdown("## Settings"),
               month_slider],
    main = [pn.Row(pn.Column(yaxis,
                             plot1.panel(width=700), margin=(0,25)), 
                 table.panel(width=500)), 
          pn.Row(pn.Column(plot2.panel(width=600), margin=(0,25)), 
                 pn.Column(yaxis2, plot3.panel(width=600)))],
    accent_base_color="#88d8b0",
    header_background="#003366",
)
# template.show()
template.servable();